In [2]:
import os

print(os.environ['DATA_DIR'])

root_data_dir = os.environ['DATA_DIR']
os.listdir(os.environ['DATA_DIR'])

/Users/username/PycharmProjects/ml_for_products/data


['bidmachine_task_data', 'bidmachine_logs.zip']

In [4]:
train_data_dir = os.path.join(root_data_dir, 'bidmachine_task_data')
os.listdir(train_data_dir)

['__MACOSX', 'train_data.csv', 'test_data.csv']

In [ ]:
from IPython.display import clear_output

!pip install catboost
clear_output()


  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 383.3 kB/s eta 0:00:00m eta 0:00:010:00:03
   ━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 9.2/20.3 MB 492.5 kB/s eta 0:00:23:24

In [8]:
import pandas as pd

train_df = pd.read_csv(os.path.join(train_data_dir, 'train_data.csv'), nrows=1000)

train_df.head()

NameError: name 'pd' is not defined

In [15]:
train_df.columns

Index(['target', 'flr', 'sellerClearPrice', 'hour', 'ssp', 'dsp',
       'auctionBidFloorSource', 'sdk', 'sdkver', 'bundle', 'os', 'lang',
       'country', 'region', 'request_context_device_w',
       'request_context_device_h', 'bidderFlrPolicy', 'adt', 'dc', 'contype',
       'request_context_device_type', 'ifa', 'price'],
      dtype='object')

In [16]:
train_df.describe()

,target,flr,sellerClearPrice,hour,ssp,dsp,request_context_device_w,request_context_device_h,contype,request_context_device_type,price
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.019000,2.525469,2.713013,11.328000,157.0,78.826000,639.792000,1247.998000,2.500000,4.205000,3.567206
std,0.136593,5.878980,4.739086,7.807007,0.0,26.458756,278.584199,555.673767,1.182962,0.403904,6.354854
min,0.000000,0.010000,0.026283,0.000000,157.0,41.000000,320.000000,568.000000,2.000000,4.000000,0.027666
25%,0.000000,0.691500,0.766361,3.000000,157.0,80.000000,390.000000,844.000000,2.000000,4.000000,1.010250
50%,0.000000,1.107266,1.521529,12.000000,157.0,80.000000,430.000000,932.000000,2.000000,4.000000,1.981744
75%,0.000000,3.105345,3.347819,18.250000,157.0,80.000000,810.000000,1600.000000,2.000000,4.000000,4.462007
max,1.000000,161.849300,124.300900,23.000000,157.0,227.000000,1812.000000,2712.000000,7.000000,5.000000,167.318500


In [11]:
train_df['target'].value_counts(normalize=True)

target
0    0.981
1    0.019
Name: proportion, dtype: float64

In [17]:
features = {
    'cat': ['request_context_device_type', 'dsp', 'ssp', 'hour'],
    'num': ['price', ]
}

In [19]:
X = train_df[features['cat'] + features['num']]
y = train_df['target']

In [ ]:
from catboost import Pool, CatBoostClassifier

from IPython.display import clear_output

model = CatBoostClassifier(**{
    'loss_function': 'CrossEntropy', 'boosting_type': 'Plain',
    'bootstrap_type': 'Bernoulli', 'n_estimators': 150
})

train_pool = Pool(data=X, label=y, cat_features=features['cat'])
model.fit(train_pool)
clear_output()
print(f'Model trained: {model}')